# Setup

## Make both T4 GPUs visiable to CUDA

In [ ]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

## Install vLLM

In [ ]:
%%time
!pip uninstall -y torch
!pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

In [ ]:
!pip install faiss-cpu sentence-transformers

## Some Configuration

In [ ]:
# In DEBUG mode, infer only on 64 problems
DEBUG = False
# Number of candidate solutions to generate
K = 16
DEPTH = 4
TEMPERATURE = 0.7
TOP_P = 0.92
BATCH_SIZE = 256

## Imports

In [ ]:
import vllm
import re
import csv
import torch
import gc
from tqdm import tqdm
import pandas as pd
from queue import Queue, Empty
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager

import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer

# Python Code Execution Environment

In [ ]:
1

## Find Python code blocks within text

In [ ]:
def find_python_blocks(text):
    blocks = re.findall(r"(```python.*?```)", text, re.DOTALL)
    # filter blocks by trying to convert them to float or int
    filtered_blocks = []
    for block in blocks:
        code = block[len("```python"):-len("```")].strip()
        try:
            x = int(code)
        except:
            filtered_blocks.append(code)
            continue
        try:
            x = float(code)
        except:
            filtered_blocks.append(code)
    return filtered_blocks        

## Class to Execute Python code (adopted from Numina)

In [ ]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout
    # handles timeout
    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()

## Execute a Python code block

In [ ]:
def execute(executor, code):
    success = False
    for lib in ("subprocess", "venv"):
        if lib in code:
            output = f"{lib} is not allowed"
            outputs.append(output)
            successes.append(success)
            continue
    try:
        success, output = executor(code)
    except TimeoutError as e:
        output = str(e)

    output = output.strip()
    
    return output, success

## Test by running some python code

In [ ]:
text = """Block 1
```python
s = 0
for i in range(100):
    s += i
print(s)
```
Block 2
```python
2**12
```
Block 3
```python
3
```
"""

In [ ]:
blocks = find_python_blocks(text)
blocks

In [ ]:
executor = PythonREPL()
outputs = [execute(executor, block) for block in blocks]
outputs

In [ ]:
for block, output in zip(blocks, outputs):
    print(f"""
```python
{block}
```
```output
{output[0]}
```"""
    )

# Load Model on vLLM

## We use the Qwen 2.5 7b Instruct Model here by Alibaba. You should explore other models.

In [ ]:
import vllm
from transformers import AutoTokenizer


# Set tensor_parallel_size to match the available GPUs
llm = vllm.LLM(
    "Qwen/Qwen2.5-32B-Instruct-AWQ",
    tensor_parallel_size=2,  # Adjust to your available GPUs
    gpu_memory_utilization=0.95,  # Slightly reduced for efficient usage
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=4096
)
tokenizer = llm.get_tokenizer()

# Utilites

## Extract boxed answer

In [ ]:
def extract_answer(text):
    # find right most boxed answer
    def last_boxed_only_string(text):
        idx = text.rfind("\\boxed")
        if idx < 0:
            idx = text.rfind("\\fbox")
            if idx < 0:
                return None
        i = idx
        right_brace_idx = None
        num_left_braces_open = 0
        while i < len(text):
            if text[i] == "{":
                num_left_braces_open += 1
            if text[i] == "}":
                num_left_braces_open -= 1
                if num_left_braces_open == 0:
                    right_brace_idx = i
                    break
            i += 1
        if right_brace_idx is None:
            return None
        return text[idx : right_brace_idx + 1]
    # get content of boxed
    def remove_boxed(boxed):
        left = "\\boxed{"
        try:
            assert boxed[: len(left)] == left
            assert boxed[-1] == "}"
            length = len(left)
            return boxed[length:-1]
        except Exception:
            return None

    boxed = last_boxed_only_string(text)
    if boxed is None:
        return None
    answer = remove_boxed(boxed)
    return answer

## Majority vote (select the most occuring answer)

In [ ]:
# Define the majority voting function to get the most common answer
def majority_vote(answers):
    answers = [answer for answer in answers if answer is not None]

    if not answers:
        return None
    # count the occurence of each answer
    counts = {}
    for answer in answers:
        if answer in counts:
            counts[answer] += 1
        else:
            counts[answer] = 1

    max_answer = None
    max_count = 0
    
    for answer, count in counts.items():
        if count > max_count:
            max_answer = answer
            max_count = count
    
    return max_answer

# RAG Utility

In [ ]:
class GeometryRAG:
    def __init__(self, geometry_dataset, encoder_model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", embedding_file="geometry_embeddings.npy"):
        self.geometry_dataset = geometry_dataset
        self.encoder = SentenceTransformer(encoder_model, device='cuda')  # Force CPU
        self.embedding_file = embedding_file
        self.index = self._build_faiss_index()

    def _build_faiss_index(self):
        if os.path.exists(self.embedding_file):
            # Load pre-computed embeddings
            embeddings = np.memmap(self.embedding_file, dtype='float32', mode='r', shape=(len(self.geometry_dataset), self.encoder.get_sentence_embedding_dimension()))
        else:
            # Compute and save embeddings
            embeddings = self.encoder.encode([item['problem'] for item in self.geometry_dataset], show_progress_bar=True)
            np.save(self.embedding_file, embeddings)
            embeddings = np.memmap(self.embedding_file, dtype='float32', mode='r', shape=embeddings.shape)

        # Build FAISS index
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)
        return index

    def retrieve_similar_problems(self, problem, k=1):
        with torch.no_grad():
            query_embedding = self.encoder.encode([problem], show_progress_bar=False)
        
        _, indices = self.index.search(query_embedding.astype('float32'), k)
        return [self.geometry_dataset[i] for i in indices[0]]

    def is_geometry_problem(self, problem):
        geometry_keywords = ['বৃত্ত', 'তল', 'কোণ', 'ত্রিভুজ', 'আয়তক্ষেত্র', 'ক্ষেত্রফল','আয়তন','বাহু','বহুভুজ', 'ব্যাসার্ধ', 'ব্যাস', 'চতুর্ভুজ','সরলরেখা','বিন্দু','অতিভুজ','সমকোণ']
        return any(keyword in problem for keyword in geometry_keywords)

    def generate_prompt(self, problem):
        if self.is_geometry_problem(problem):
            similar_problems = self.retrieve_similar_problems(problem)
            context = "\n\n".join([f"Similar problem: {p['problem']}\nSolution: {p['solution']}" for p in similar_problems])
            
            return f"""Here is a math problem in Bengali:
{problem}

Here are some similar geometry problems and their solutions for reference:
{context}

The answer is a non-negative integer. Please reason step by step to solve the problem above, using the similar problems as guidance if helpful. Provide python code to verify your reasoning.
Put your final integer answer within \\boxed{{}}.
Remember, you must provide the Python code, even if you're unsure about the answer."""
        else:
            return f"""Here is a math problem in Bengali:
{problem}
The answer is a non-negative integer. Please reason step by step to solve the problem above. Provide python code to verify your reasoning.
Put your final integer answer within \\boxed{{}}.
Remember, you must provide the Python code, even if you're unsure about the answer."""

In [ ]:
geometry_prblms = pd.read_csv("/kaggle/input/numinamath-tir-bn/numina-math-tir-bn.csv")
geometry_prblms.head()

In [ ]:
geometry_dataset=geometry_prblms.to_dict('records')
geometry_dataset[:2]

In [ ]:
rag_system=GeometryRAG(geometry_dataset)

# TIR Agent

In [ ]:
class TIRAgent:
    def __init__(self, problem_id, id, problem, tokenizer, max_depth, log,rag_system):
        # problem id
        self.problem_id = problem_id
        # id of the agent
        self.id = id
        # number of LLM turns
        self.depth = 1
        # maximum number of turns allowed
        self.max_depth = max_depth
        # LLM's tokenizer
        self.tokenizer = tokenizer
        # Problem statement
        self.problem = problem
        #RAG System
        self.rag_system=rag_system
        # Chat Messages
        self.messages = [
            {
                "role":"user",
                "content":self.rag_system.generate_prompt(self.problem)
            }
        ]
        # Last response from the LLM
        self.last_response = None
        # Code blocks from the last response
        self.blocks = []
        # Answers that the LLM generated in \boxed{}
        self.answers = []
        # No python code generated in last response or max_depth reached
        self.is_complete = False
        # File to log answers
        self.log = log
        # Next prompt to the LLM
        self.next_prompt = None
        
    def complete(self):
        # is the Agent done
        return self.is_complete
    
    def add_response(self, response, executor):
        self.depth += 1
        # remember this response
        self.last_response = response
        # add this to the messages history
        self.messages.append({"role": "assistant", "content": response})
        # extract python blocks
        self.blocks = find_python_blocks(response)
        # extract answer from the generated text, if present
        answer = extract_answer(response)
        if answer is not None:
            self.answers.append(answer)
        # is it done?
        self.is_complete = not self._should_continue()
        # if not, use the python executor to create next prompt
        if not self.is_complete:
            self.next_prompt = self._next_prompt(executor)   
            self.messages.append({"role": "user", "content": self.next_prompt})
    
    def _should_continue(self):        
        # quit if max_depth number of turns reached
        if self.depth >= self.max_depth:
            return False
        # if no python code generated, we can stop now
        elif len(self.blocks) > 0:
            return True
        return False
    
    def _next_prompt(self, executor):
        assert not self.is_complete
        assert len(self.blocks) > 0
        # get code result from python execution
        output, status = execute(executor, self.blocks[-1])
        
        prompt = ''
        # if code succeeds give the output
        if status:
            prompt = f"""The python code you provided gives the following output:
```python
{self.blocks[-1]}
```
```output
{output}
```"""
        # if code fails, give the error
        else:
            prompt = f"""The python code you provided gives the following error:
```python
{self.blocks[-1]}
```
```output
{output}
```"""
        return prompt
    
    
    def next_message(self):
        assert not self.is_complete 
        # apply chat template to get the text
        text = self.tokenizer.apply_chat_template(
            self.messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        return text
        
    
    def final_answer(self):
        # if there no answers yet, we have to return None
        ans = None
        # otherwise return the latest answer
        if len(self.answers) > 0:
            ans = self.answers[-1]
        # log to file
        if self.log:
            self.log.writerow([self.problem_id, self.id, ans])
        # try to convert to integer
        try:
            ans = int(ans)
        except:
            ans = None
        
        return ans        

# Sc-TIR Agent

In [ ]:
class SCTIRAgent:
    def __init__(self, problem_id, problem, tokenizer, samples, max_depth, log,rag_system):
        # problem id
        self.problem_id = problem_id
        # problem statement
        self.problem = problem
        # LLM's tokenizer
        self.tokenizer = tokenizer
        # number of TIRAgents to create
        self.samples = samples
        # maximum number of turns
        self.max_depth = max_depth
        #RAG System
        self.rag_system=rag_system
        # TIR Agents
        self.agents = [TIRAgent(problem_id, i, problem, tokenizer, max_depth, log,rag_system) for i in range(samples)]
        # log file
        self.log = log
    
    def complete(self):
        # only complete when all agents are done
        for agent in self.agents:
            if not agent.complete():
                return False
        return True
        
    def get_ready_agents(self):
        # return agents that are not complete yet
        ready_agents = []
        for agent in self.agents:
            if not agent.complete():
                ready_agents.append(agent)
        return ready_agents
    
    def final_answer(self):
        # majority vote agent answers
        assert self.complete()
        answers = [agent.final_answer() for agent in self.agents]
        answer = majority_vote(answers)
        if answer is None:
            return 0
        return answer

# Load Test Set

In [ ]:
test_df = pd.read_csv('/kaggle/input/dlsprint3/test.csv')
test_df.sample(5)

## Load 5 problems since we are short on time

In [ ]:
if DEBUG:
    test_df = test_df[:5]
    torch.cuda.empty_cache()
    gc.collect()

# Configure LLM and Python REPL

In [ ]:
sampling_params = vllm.SamplingParams(max_tokens=2048, temperature=TEMPERATURE, top_p=TOP_P)
executor = PythonREPL()

# Run the Agents

## TIR Agent

In [ ]:
# for row in test_df.values[20:21]:
#     problem_id = row[0]
#     problem = row[1]
    
#     agent = TIRAgent(problem_id, 0, problem, tokenizer, max_depth=4, log=None,rag_system=rag_system)
    
#     while not agent.complete():
#         text = agent.next_message()
#         # get response from LLM
#         response = llm.generate([text], sampling_params)
#         # pass in python executor, since response might contain python code
#         agent.add_response(response[0].outputs[0].text, executor)
    
#     for message in agent.messages:
#         print(f"Role: {message['role']}\n")
#         print(f"Content:\n {message['content']}\n")
    
#     answer = agent.final_answer()
#     print(f"Final answer: {answer}")
    

## SC-TIR Agent

In [ ]:
# for row in test_df.values[20:21]:
#     problem_id = row[0]
#     problem = row[1]
    
#     agent = SCTIRAgent(problem_id, problem, tokenizer, samples=4, max_depth=4, log=None,rag_system=rag_system)
    
#     while not agent.complete():
#         ready_agents = agent.get_ready_agents()
#         texts = [a.next_message() for a in ready_agents]
#         # get response from LLM
#         responses = llm.generate(texts, sampling_params)
#         # pass response to the agents
#         for i, ready_agent in enumerate(ready_agents):
#             ready_agent.add_response(responses[i].outputs[0].text, executor)
    
#     answer = agent.final_answer()
#     print(f"Problem: {problem}")
#     print(f"Final answer: {answer}")

# Run Inference

# Create submission

## Also log agent answers

In [ ]:
file = open('submission.csv', 'w', encoding='utf-8')
log_file = open('log.csv', 'w', encoding='utf-8')

submission = csv.writer(file)
log = csv.writer(log_file)

submission.writerow(['ID', 'Answer'])
log.writerow(['ID', "Agent ID", 'Answer'])

## Configure LLM sampling parameters and Python REPL

## Use a queue to Batch inference

In [ ]:
%%time

boxed_answers = {}
agents = []

q = Queue()

iterator = iter(tqdm(test_df.values))

while True:
    for agent in agents:
        if agent.complete():
            boxed_answers[agent.problem_id] = agent.final_answer()

    agents[:] = list(filter(lambda a: not a.complete(), agents))

    while q.qsize() < BATCH_SIZE:
        try:
            row = next(iterator)
        except StopIteration:
            break

        id = row[0]
        problem = row[1]

        agent = SCTIRAgent(id, problem, tokenizer, K, DEPTH, log,rag_system)
        
        agents.append(agent)

        for tir_agent in agent.get_ready_agents():
            q.put_nowait(tir_agent)
            
    if q.empty():
        break
        
    
    ready_agents = []
    texts = []
    for _ in range(BATCH_SIZE):
        try:
            agent = q.get_nowait()
            ready_agents.append(agent)
            texts.append(agent.next_message())
        except:
            break

    
    responses = llm.generate(texts, sampling_params)
    responses = [response.outputs[0].text for response in responses]
    
    for i in range(len(ready_agents)):
        agent = ready_agents[i]
        response = responses[i]
        agent.add_response(response, executor)
        if not agent.complete():
            q.put_nowait(agent)
   

## Write to submission file

In [ ]:
for id, answer in boxed_answers.items():
    submission.writerow([id, answer])

## Close files

In [ ]:
   
file.close()
log_file.close()

In [ ]:
submit_df=pd.read_csv("/kaggle/working/submission.csv")
submit_df.head()

In [ ]:
submit_df.sort_values(by=['ID'],inplace=True,ascending=True)
submit_df.head()

In [ ]:
submit_df.to_csv("/kaggle/working/submission.csv")